In [1]:
import spacy
import os
import pandas as pd
import numpy as np
import re
import nltk


In [2]:
nlp = spacy.load("ru_core_news_lg")
# nlp = spacy.load("ru_core_news_sm")

In [3]:
try:
    #локальный адрес
    data = pd.read_csv(os.path.abspath(os.path.curdir)+'\\test_data.csv')
except:
    #адрес в другом месте
    data = pd.read_csv(os.path.abspath(os.path.curdir)+'/test_data.csv')
    

In [4]:
# Столбцы для парсинга реплик
data[['greeting', 'said_goodbye', 'name_manager', 'client_company','manager_introduced_himself']]=0

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   dlg_id                      480 non-null    int64 
 1   line_n                      480 non-null    int64 
 2   role                        480 non-null    object
 3   text                        480 non-null    object
 4   greeting                    480 non-null    int64 
 5   said_goodbye                480 non-null    int64 
 6   name_manager                480 non-null    int64 
 7   client_company              480 non-null    int64 
 8   manager_introduced_himself  480 non-null    int64 
dtypes: int64(7), object(2)
memory usage: 33.9+ KB


In [6]:
# Уникальные диалоги
data['dlg_id'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [7]:
# Диалоги менеджера

In [8]:
for dlg_id in range(6):# Цикл по диалогам
    text_manager=data.loc[(data['role'] =='manager')&(data['dlg_id']==dlg_id) ,['text']]

    for index in text_manager.index:# цикл по индексам реплик менеджера
        doc=nlp(text_manager.loc[index,'text'].lower())# храним обработанную реплику диалога
        #приветсвие
        for token in doc:# цикл по токенам
            if token.head.text=='день' and token.dep_ =='amod':
                print(token.text,token.head.text)
                data.loc[index,'greeting']=1
            elif token.head.text=='здравствуйте' and token.dep_ =='ROOT':
                print(token.head.text)
                data.loc[index,'greeting']=1
            elif token.head.text=='добрый' and token.dep_ =='ROOT':
                print(token.head.text)
                data.loc[index,'greeting']=1
        #прощание
        for token in doc:# цикл по токенам
            if token.head.text=='свидания' and token.dep_ =='case':
                print(token.text, token.head.text)
                data.loc[index,'said_goodbye']=1
        #Менеджер представился
        for token in doc:
            if token.head.text=='зовут' and token.ent_type != 0:
                print(token.text)
                data.loc[index,'manager_introduced_himself']=1
            
    

добрый день
здравствуйте
здравствуйте
добрый день
до свидания
добрый
до свидания
здравствуйте


In [9]:
#Реплика приветсвия менеджера
print(data.loc[(data['greeting']==1) ,['text']])

                                                  text
2                                          Добрый день
109                Да здравствуйте когда заканчивается
165                                       Здравствуйте
249                                        Добрый день
304                                             Добрый
358  Да да да здравствуйте анастасия меня слышно да...


In [10]:
#Реплика прощания менеджера
print(data.loc[(data['said_goodbye']==1) ,['text']])

                   text
301      Да до свидания
336  Угу да до свидания


In [11]:
#Реплика где менеджер представился
print(data.loc[(data['manager_introduced_himself']==1),['text']])

Empty DataFrame
Columns: [text]
Index: []


In [12]:
# Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
dlg_id_greeting=(data.loc[(data['greeting']==1) ,['dlg_id']])
dlg_id_goodbay=(data.loc[(data['said_goodbye']==1) ,['dlg_id']])


In [13]:
dlg_id_greeting

,dlg_id
2,0
109,1
165,2
249,3
304,4
358,5


In [14]:
dlg_id_goodbay

,dlg_id
301,3
336,4


In [15]:
merged = dlg_id_greeting.merge(dlg_id_goodbay, indicator=True, how='outer')

In [16]:
merged

,dlg_id,_merge
0,0,left_only
1,1,left_only
2,2,left_only
3,3,both
4,4,both
5,5,left_only


In [17]:
print('Диалоги в которых выполняется требования к менеджеру')
display (merged[merged['_merge'] == 'both'])

Диалоги в которых выполняется требования к менеджеру


,dlg_id,_merge
3,3,both
4,4,both


In [18]:
# Или можно так
# dlg_id_goodbay.reset_index(inplace=True,drop=True)
# dlg_id_greeting.reset_index(inplace=True,drop=True)

In [19]:
# print('Диалоги в которых выполняется требования к менеджеру')
# for i in range(len(dlg_id_greeting)):
#     a=dlg_id_greeting.loc[i,'dlg_id']
#     for j in range(len(dlg_id_goodbay)):
#         b=dlg_id_goodbay.loc[j,'dlg_id']
#         if a==b:
#             print(a)
                

In [20]:
# Диалоги клиента

In [21]:
for dlg_id in range(6):# Цикл по диалогам
    text_manager=data.loc[(data['role'] =='client')&(data['dlg_id']==dlg_id) ,['text']]

    for index in text_manager.index:# цикл по индексам реплик менеджера
        doc=nlp(text_manager.loc[index,'text'].lower())# храним обработанную реплику диалога
        # Имя компании
        for token in doc:
            if token.head.text=='компания' and token.dep_=='appos':
                print(token.head.text,token.text)
                data.loc[index,'client_company']=token.text
            else:
                if token.text=='компания' and token.dep_=='nsubj':
                    print(token.head.text,token.text)
                    data.loc[index,'client_company']=token.head.text
    
        # Имя менеджера
        
        for token in doc:
            if token.head.text=='алло' and token.dep_ =='appos':
                print(token.text)
                data.loc[index,'name_manager']=token.text
        
        # Имя клиента
#         for token in doc:
#             if token.head.text=='зовут' and token.ent_type != 0:
#                 print(token.text)                

диджитал компания
диджитал компания
диджитал компания
дмитрий
компания китобизнес


In [22]:
# Имя менеджера
data.loc[(data['role'] =='client')&(data['name_manager']!=0),['name_manager'] ]

,name_manager
250,дмитрий


In [23]:
# Название компании
data.loc[(data['role'] =='client')&(data['client_company']!=0),['client_company'] ]

,client_company
3,диджитал
111,диджитал
167,диджитал
251,китобизнес


In [24]:
# Разбор реплик
# def text_nlp(doc):
#     print([w.text for w in doc])
#     print()
#     for token in doc:
#         print(token.text, token.pos_, token.dep_)
#     print()
#     for token in doc:    
#         print(token.head.text, token.dep_, token.text,token.ent_type)
#     print()
#     for sent in doc.sents:
#         print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'pobj'])
#     print()
#     for token in doc:
#         if token.ent_type != 0:
#             print(token.text, token.ent_type_)

In [25]:
# Синонимы только английские
# from nltk.corpus import wordnet

# synonyms = []

# for syn in wordnet.synsets("life"):
#     for lm in syn.lemmas():
#              synonyms.append(lm.name())
# print (set(synonyms))